In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:13 https://developer.

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-02-27 22:42:11--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.68MB/s    in 0.6s    

2022-02-27 22:42:13 (1.68 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
# Start Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

**Extract the Dataset You Used in Deliverable 1**

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Jewelry_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   50423057|R135Q3VZ4DQN5N|B00JWXFDMG|     657335467|Everbling Purple ...|         Jewelry|          5|            0|          0|   N|                Y|           Beauties!|so beautiful even...| 2015-08-31|
|         US|   11262325|R2N0QQ6R4T7YRY|B00W5T1H9W|      26030170|925 Sterling Silv...|         Jewelry|          5|    

**Pick Reviews Greater Than or Equal to 20**

In [6]:
# Filter Data, Create DataFrame, Retrieve 'total_votes' equal to or greater than 20
vine_df = df.filter("total_votes >= 20")
vine_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18238567|R16YC6SMGKA8SR|B00MD99R9G|     692935467|Fun Daisy New Des...|         Jewelry|          5|           23|         23|   N|                Y|Beautiful stateme...|This is the first...| 2015-08-31|
|         US|   35346149| R5O4WE9UM60B0|B00Q8BYTAW|      48877050|Women's New Simpl...|         Jewelry|          1|    

**Filter Data Frame, Create New, Retrieve 'helpful_votes' Divided By 'total_votes' is => 50%**

In [7]:
# Filter/New DF
helpful_vine_df = vine_df.filter("helpful_votes/total_votes>=0.5")
helpful_vine_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18238567|R16YC6SMGKA8SR|B00MD99R9G|     692935467|Fun Daisy New Des...|         Jewelry|          5|           23|         23|   N|                Y|Beautiful stateme...|This is the first...| 2015-08-31|
|         US|   35346149| R5O4WE9UM60B0|B00Q8BYTAW|      48877050|Women's New Simpl...|         Jewelry|          1|    

**Retrieve All Rows Where Review was Written as part of the Vine program**

In [8]:
# Filter/New DF, vine=='Y'
vine_review_df = helpful_vine_df.filter("vine == 'Y'")
vine_review_df.show(30)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51635935|R15PXA9XU9ZKSU|B00TDNZV4A|     280424829|Sterling Silver C...|         Jewelry|          4|           29|         29|   Y|                N|So classy and gor...|You can wear thes...| 2015-05-09|
|         US|   52833354|R2LJ4QYQ4PYDEH|B00TDNZUYG|     280424829|Sterling Silver C...|         Jewelry|          5|    

In [9]:
vine_review_df.count()

21

In [10]:
# Filter/New Df, vine == 'N'
nonvine_review_df = helpful_vine_df.filter("vine == 'N'")
nonvine_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18238567|R16YC6SMGKA8SR|B00MD99R9G|     692935467|Fun Daisy New Des...|         Jewelry|          5|           23|         23|   N|                Y|Beautiful stateme...|This is the first...| 2015-08-31|
|         US|   35346149| R5O4WE9UM60B0|B00Q8BYTAW|      48877050|Women's New Simpl...|         Jewelry|          1|    

In [11]:
nonvine_review_df.count()

7689

**Determine total # of reviews, # of 5-star reviews, and % of 5-star reviews for paid vs unpaid**

In [12]:
# Total number of helpful reviews
helpful_vine_df.count()

7710

In [15]:
# Total number of 5-star reviews (data check)
five_star_df = helpful_vine_df.filter("star_rating == '5'")
five_star_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18238567|R16YC6SMGKA8SR|B00MD99R9G|     692935467|Fun Daisy New Des...|         Jewelry|          5|           23|         23|   N|                Y|Beautiful stateme...|This is the first...| 2015-08-31|
|         US|    4039595|R2OKV47GETH0L7|B00VUTQVDK|     939458680|Jewelrieshop Stai...|         Jewelry|          5|    

In [16]:
# Total helpful 5 star (data-check)
five_star_df.count()

4455

In [17]:
# Total 5-star from non-Vine reviews
nonvine_fivestar_df = nonvine_review_df.filter("star_rating == '5'")
nonvine_fivestar_df.count()

4444

In [19]:
# 5-star reviews and unpaid
percentage_unpaid = (nonvine_fivestar_df.count() / nonvine_review_df.count()) * 100
print(str(round(percentage_unpaid, 2)) + "% of unpaid reviews with 5-stars")

57.8% of unpaid reviews with 5-stars


In [22]:
# Total 5 star from Vine reviews
vine_fivestar_df = vine_review_df.filter("star_rating == '5'")
vine_fivestar_df.count()

11

In [21]:
# 5-star reviews and paid
percentage_paid = (vine_fivestar_df.count() / vine_review_df.count()) * 100
print(str(round(percentage_paid, 2)) + "% of paid reviews with 5-stars")

52.38% of paid reviews with 5-stars
